<a href="https://colab.research.google.com/github/Srivardini11/lark/blob/main/Flkr_g_15_20k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

In [1]:
!pip install lightkurve --quiet

     |████████████████████████████████| 247 kB 4.2 MB/s 
     |████████████████████████████████| 4.5 MB 55.6 MB/s 
     |████████████████████████████████| 98 kB 7.2 MB/s 
     |████████████████████████████████| 41 kB 175 kB/s 
     |████████████████████████████████| 832 kB 62.8 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 3.6 MB 44.9 MB/s 


# **Import**

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib 
import lightkurve as lk
import numpy as np
from astropy.convolution import convolve, Box1DKernel
import seaborn as sb

matplotlib.rcParams['figure.figsize'] = (15,8)
sb.set_style('darkgrid')

# code

In [6]:
df = pd.read_csv('kepler_quarter_9.csv')

In [7]:
df_test = df[15000:20000]



In [19]:
from google.colab import files

In [20]:
flr_list = []
logg_list = []
for id in df_test['Dataset Name']:
  name    = id[:13]
  try:
    source  = lk.search_lightcurve(name, author="Kepler", cadence="long",quarter = 9).download()
  except :
    print("Crashed!")
    sample=pd.DataFrame(zip(flr_list, df_test['Log G'].values), columns=['Flkr','Log G'])

    s2 = sample[sample['Flkr']<0.1]
    s2.to_csv('data.csv')
    flr = sample['Flkr']
    flr.to_csv('Flicker.csv')
    files.download('Flicker.csv')
    files.download('data.csv')
    break 
  rem_out = source.remove_outliers(sigma_upper = 2.5).normalize()

  pg      = rem_out.to_periodogram(normalization='psd')
  seismo  = pg.flatten().to_seismology() 
  seismo.estimate_numax() 
  logg = seismo.estimate_logg()

  logg_list.append(logg.value)   


  index=50
  ind=250            
  flux    = rem_out.flux.value
  time    = rem_out.time.value
  smoothed_signal = convolve(flux, Box1DKernel(16))
  flkr    = flux[index:-ind]-smoothed_signal[index:-ind]
  flr     = np.sqrt(np.mean(flkr**2))*1e3

  flr_list.append(flr)

Crashed!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
flr = sample['Flkr']
flr

0       0.426933
1       0.475875
2       0.553015
3       0.183268
4       0.096194
          ...   
2053    0.134440
2054    0.294707
2055    0.322387
2056    0.190357
2057    0.967200
Name: Flkr, Length: 2058, dtype: float64

In [ ]:
flkr_g=pd.DataFrame(zip(flr_list, df_test['Log G'].values), columns=['Flkr','Log G'])
flkr_g_pone = flkr_g[flkr_g['Flkr']<0.1]
flkr_g_pone

In [ ]:
flkr_g_pone.to_csv('f8_g_15kto20k_<0.1.csv')

In [ ]:
df_test['flicker']= flr_list

In [ ]:
flkr_g.to_csv('f8_g_15kto20k.csv')
flkr_g

In [ ]:
x = df_test['flicker']
y = df_test['Log G']

plt.scatter(x,y)
plt.xlim(0,2)
plt.ylim(5.2, 2)

In [ ]:
x = flkr_g['Flkr']
y = flkr_g['Log G']

plt.scatter(x,y)
plt.xlim(0,.12)
plt.ylim(5,3.5)